In [12]:
import sqlalchemy as sa
import sqlalchemy.orm as orm
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import scoped_session, sessionmaker

Session = scoped_session(sessionmaker())
engine = sa.create_engine('sqlite://', echo=False)
Session.configure(bind=engine)

Base = declarative_base(bind=engine)


class User(Base):
    __tablename__ = "User"

    id = sa.Column(sa.Integer, primary_key=True)
    group_id = sa.Column(sa.Integer, sa.ForeignKey("Group.id"))
    group = orm.relationship("Group", backref="users", uselist=False)
    name = sa.Column(sa.String(255), default="", nullable=False)


class Group(Base):
    __tablename__ = "Group"
    id = sa.Column(sa.Integer, primary_key=True)
    name = sa.Column(sa.String(255), default="", nullable=False)


Base.metadata.create_all()
g = Group(name="A")
g.users.append(User(name="x"))
g.users.append(User(name="y"))
g.users.append(User(name="z"))
Session.add(g)
g2 = Group(name="B")
g2.users.append(User(name="i"))
g2.users.append(User(name="j"))
g2.users.append(User(name="k"))
Session.add(g2)
Session.commit()


In [13]:
import pandas as pd
pd.read_sql

<function pandas.io.sql.read_sql>

In [14]:
query = Session.query(Group)
df = pd.read_sql(query.statement, query.session.bind)
df

,id,name
0,1,A
1,2,B


In [30]:
query = Session.query(User)
df = pd.read_sql(query.statement, query.session.bind)
df

,id,group_id,name
0,1,1,x
1,2,1,y
2,3,1,z
3,4,2,i
4,5,2,j
5,6,2,k


In [19]:
g = Session.query(Group).first()
query = Session.query(User).filter(User.group_id == g.id)
df = pd.read_sql(query.statement, query.session.bind)
df

,id,group_id,name
0,1,1,x
1,2,1,y
2,3,1,z


In [29]:
query = Session.query(User).join(Group, User.group_id == Group.id)
query = query.with_entities(User.id, User.name, Group.id.label("group_id"), Group.name.label("group_name"))
df = pd.read_sql(query.statement, query.session.bind)
df


,id,name,group_id,group_name
0,1,x,1,A
1,2,y,1,A
2,3,z,1,A
3,4,i,2,B
4,5,j,2,B
5,6,k,2,B


In [32]:
query = Session.query(User)
df = pd.read_sql(query.statement, query.session.bind)
df.ix[:, ["id", "name"]]

,id,name
0,1,x
1,2,y
2,3,z
3,4,i
4,5,j
5,6,k


In [57]:
query = Session.query(User)
df = pd.read_sql(query.statement, query.session.bind)
df.groupby("group_id").agg({"name": lambda x: ", ".join(x)})

,name
group_id,
1,"x, y, z"
2,"i, j, k"
